# ML441 Assignment 4
## Charles de Kock - 26023830

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

## Data processing

In [5]:
og_data = pd.read_csv('../data/breastCancer.csv', delimiter='\t', na_values='?')

In [6]:
og_data['diagnosis'] = og_data['diagnosis'].map({'B': 0, 'M': 1})
og_data = og_data.drop(columns=['gender', 'id'])

In [25]:
og_data.describe(include='all')

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Bratio
count,569.000000,563.000000,564.00000,567.000000,563.000000,567.000000,565.000000,562.000000,567.000000,563.000000,...,561.000000,566.000000,563.000000,566.000000,563.000000,564.000000,567.000000,567.000000,569.000000,569.000000
mean,0.372583,14.091401,19.23773,91.972892,655.373535,176.164990,0.104411,0.088318,0.048734,0.181252,...,25.689305,107.089523,883.312256,0.132483,0.254414,0.272365,0.114330,0.290347,0.083946,0.496699
std,0.483918,3.513903,4.61416,24.341236,352.453278,4192.503875,0.052897,0.079175,0.038685,0.027474,...,6.143973,33.345818,571.501837,0.022831,0.157792,0.209305,0.065682,0.061801,0.018061,0.299466
min,0.000000,6.981000,-19.98000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000049
25%,0.000000,11.685000,16.16750,75.100000,420.300000,0.086210,0.064920,0.029240,0.020310,0.162000,...,21.100000,84.087500,515.550000,0.116650,0.146600,0.113925,0.064530,0.250550,0.071460,0.238391
50%,0.000000,13.280000,18.85500,86.180000,551.100000,0.095920,0.092630,0.061400,0.033410,0.179300,...,25.410000,97.655000,686.600000,0.131400,0.211900,0.226550,0.099750,0.282300,0.080040,0.501432
75%,1.000000,15.750000,21.81250,104.200000,785.600000,0.105400,0.130400,0.130350,0.073820,0.195700,...,29.660000,125.325000,1091.500000,0.146075,0.339500,0.385300,0.161300,0.318150,0.092080,0.764796
max,1.000000,28.110000,39.28000,188.500000,2501.000000,99831.000000,0.345400,0.426800,0.201200,0.304000,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,0.999741


In [26]:
og_data.isna().sum().sum()

np.int64(127)

In [42]:
correlation_matrix = og_data.corr()

In [51]:
high_corr_mask = (correlation_matrix.abs() > 0.95) & (correlation_matrix.abs() < 1.0)
highly_correlated_features = high_corr_mask.any(axis=0)
highly_correlated_features = highly_correlated_features[highly_correlated_features]

In [52]:
highly_correlated_features

radius_mean        True
perimeter_mean     True
area_mean          True
radius_se          True
perimeter_se       True
area_se            True
radius_worst       True
perimeter_worst    True
area_worst         True
dtype: bool

In [50]:
correlation_matrix['radius_se']

diagnosis                  0.562160
radius_mean                0.678640
texture_mean               0.273173
perimeter_mean             0.694782
area_mean                  0.735702
smoothness_mean            0.054183
compactness_mean           0.500684
concavity_mean             0.630732
concave points_mean        0.697214
symmetry_mean              0.302250
fractal_dimension_mean    -0.001963
radius_se                  1.000000
texture_se                 0.209841
perimeter_se               0.972837
area_se                    0.952171
smoothness_se              0.163653
compactness_se             0.338807
concavity_se               0.333066
concave points_se          0.517229
symmetry_se                0.240020
fractal_dimension_se       0.230155
radius_worst               0.718563
texture_worst              0.188665
perimeter_worst            0.722088
area_worst                 0.756437
smoothness_worst           0.145638
compactness_worst          0.288789
concavity_worst            0

In [16]:
def find_outliers_iqr(df, thresh=1.5):
    outlier_counts = {}
    for col in df.select_dtypes(include=['number']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - thresh * IQR
        upper_bound = Q3 + thresh * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
        outlier_counts[col] = len(outliers)
    
    for col, count in outlier_counts.items():
        print(f"{col}: {count}")
    
    return outlier_counts

In [9]:
def detect_outliers_iqr(df):
    Q1 = df.select_dtypes(include=[np.number]).quantile(0.25)
    Q3 = df.select_dtypes(include=[np.number]).quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (df < lower_bound) | (df > upper_bound)

def handle_outliers_iqr(df, method='cap', threshold=1.5):
    df_clean = df.copy()
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    outliers_mask = detect_outliers_iqr(df[numeric_cols])
    
    if method == 'cap':
        for col in numeric_cols:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            df_clean[col] = np.clip(df[col], lower_bound, upper_bound)
    
    elif method == 'transform_log':
        df_clean[numeric_cols] = np.log1p(df[numeric_cols])
    
    return df_clean

In [10]:
data_clean = handle_outliers_iqr(og_data, method='transform_log')

/Users/charliedekock/Desktop/Machine_Learning/ml_env/lib/python3.11/site-packages/pandas/core/internals/blocks.py:395: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


In [23]:
temp_outliers = find_outliers_iqr(og_data, thresh=5)
out = 0
for k in list(temp_outliers.keys()):
    out += temp_outliers[k]
print(out)

diagnosis: 0
radius_mean: 0
texture_mean: 1
perimeter_mean: 0
area_mean: 0
smoothness_mean: 1
compactness_mean: 0
concavity_mean: 0
concave points_mean: 0
symmetry_mean: 0
fractal_dimension_mean: 0
radius_se: 2
texture_se: 1
perimeter_se: 2
area_se: 5
smoothness_se: 2
compactness_se: 1
concavity_se: 2
concave points_se: 1
symmetry_se: 1
fractal_dimension_se: 4
radius_worst: 0
texture_worst: 0
perimeter_worst: 0
area_worst: 1
smoothness_worst: 0
compactness_worst: 0
concavity_worst: 0
concave points_worst: 0
symmetry_worst: 1
fractal_dimension_worst: 1
Bratio: 0
26


In [21]:
temp_clean = find_outliers_iqr(data_clean, thresh=4)

out = 0
for k in list(temp_clean.keys()):
    out += temp_clean[k]
print(out)

diagnosis: 0
radius_mean: 0
texture_mean: 0
perimeter_mean: 0
area_mean: 0
smoothness_mean: 1
compactness_mean: 0
concavity_mean: 0
concave points_mean: 0
symmetry_mean: 0
fractal_dimension_mean: 0
radius_se: 2
texture_se: 0
perimeter_se: 0
area_se: 0
smoothness_se: 4
compactness_se: 1
concavity_se: 2
concave points_se: 1
symmetry_se: 3
fractal_dimension_se: 4
radius_worst: 0
texture_worst: 0
perimeter_worst: 0
area_worst: 0
smoothness_worst: 0
compactness_worst: 0
concavity_worst: 0
concave points_worst: 0
symmetry_worst: 1
fractal_dimension_worst: 1
Bratio: 0
20
